In [1]:
import pandas as pd
import random
import itertools
import numpy as np
import time 
import pickle
from scipy.stats import ttest_ind, chi2_contingency, fisher_exact, ranksums

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Context and Background

In pancreatic cancer, there are two main subtypes that have been identified: 'basal-like' and 'classical.' However, the problem is that we don't really know which gene to target in 'basal-like' subtypes and which to target in 'classical' subtypes so they're functionally useless because we don't know how to treat each uniquely. Therefore, I think it could be interesting to perform a correlation analysis between each of the genes in our lists (both the FDA and also the novel drug list) and see if any of these genes associate strongly with basal-like or classical signature strength.

So, more precisely, we are assessing correlation between GENE X's expression values and signature scores for basal-like/classical across all cells that contain non-zero values for each. Do this only on tumor cells, we don't care about the other cell types. The non-zero expression value of gene X should be pretty straightforward; this is already present in the single cell matrix. To determine the signature score, we will use the same formula as before: 

**Equation:**
Signature Mean - Background Mean = Signature Score 

For each cell, there will be a unique Signature Mean and a unique Background Mean, and therefore a unique Signature Score for each cell.

Calculating Signature Mean:
Extract the specific columns that line up with the gene names in the signature (see each attachment separately).
Take the average value of all of those column values for that one cell/row. You do not take the average of values down a column in this case.

Use the lists attached for basal-like (Moffitt basal-like) and classical like (Moffitt classical-like) to calculate a basal-like signature mean and also a classical signature mean for each cell (you have already computed a background score in the past, so you can just use that same unique value from before for each cell). Then subtract the background mean from the signature mean to get the signature score for each cell. Then similar to the analysis in Part #4, let's generate a list of genes that have the strongest correlations with basal-like scores and then separately also a list that correlates with classical signature scores (using both pearson and spearman, p and r). 

In [8]:
%cd drive/My\ Drive/Broad

/content/drive/My Drive/Broad


In [9]:
%pwd

'/content/drive/My Drive/Broad'

In [13]:
subset = 'fda'
gene_file = '110920_fda drug list v1.csv'

In [10]:
column_names = pd.read_csv('data/columns for X.csv',header = None)
column_names = list(column_names[0])

In [11]:
rows = pd.read_csv('data/rows for X.csv',header = None)
cell_type = list(rows[0])
cell_type = [val.lower() for val in cell_type]

In [31]:
skip_rows = [idx  for idx, val in enumerate(cell_type) if val != 'tumor']

In [15]:
gene_list = pd.read_csv('data/{}'.format(gene_file), header = None)
gene_list = list(gene_list[0])

In [20]:
groups_list = [
    ('basal','data/Moffitt Basal-like.csv'),
    ('classifical','data/Moffit Classical-like.csv'),
]

In [21]:
gene_group_list = []
gene_group_missing_list = []

for name,group_file in groups_list:
    print(name)
    group_genes = pd.read_csv(group_file, header = None)
    group_genes = list(group_genes[0])
    
    group_genes_cols = [gene for gene in group_genes if gene in column_names]
    group_genes_missing = [gene for gene in group_genes if gene not in column_names]

    gene_group_list.append((name,group_genes_cols))
    gene_group_missing_list.append((name,group_genes_missing))

basal
classifical


In [35]:
def get_background(row):
    mean_list = []
    n_cols = len(row)    
    for i in range(25):
        col_rand = random.sample(range(0, n_cols), 50)
        row_rand = row.iloc[col_rand]
        mean_list.append(np.nanmean(row_rand))
        
    return np.nanmean(mean_list)

In [ ]:
df_X = pd.read_csv('data/X.csv',names = column_names, skiprows = skip_rows)

In [ ]:
df_X.shape